In [1]:
import itertools

mer_size = 4

all_mers = ["".join(x) for x in itertools.product("ACTG", repeat=mer_size) ]
all_mers[:3]

['AAAA', 'AAAC', 'AAAT']

In [2]:
from Bio.Seq import reverse_complement

without_complement = set([])
for c in all_mers:
    if c not in without_complement and reverse_complement(c) not in without_complement:
        without_complement.add(c)


len(without_complement)

136

In [3]:
import numpy as np


def count_mers(sequence):
    mer_counts = { mer: 0 for mer in all_mers}
    for i in range(len(sequence) - mer_size + 1):
        mer = sequence[i: i + mer_size]
        mer_counts[mer] += 1

    return mer_counts

def simplify_counts(counts):
    simplified_counts = { mer: 0 for mer in without_complement}
    for mer, count in counts.items():
        if mer in without_complement:
            simplified_counts[mer] += count
        else:
            simplified_counts[ reverse_complement(mer) ] += count 
    
    return simplified_counts

def get_seq_embedding(sequence):
    counts = count_mers(sequence)
    simplified = simplify_counts(counts)
    embeddings = np.array([v for v in counts.values()])
    return embeddings / len(sequence)


get_seq_embedding("ACTGA")
get_seq_embedding("ACTGG")

array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0.2, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.2, 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.

In [4]:
def read_data(file_path):
    with open(file_path, 'r') as f:
        sequences = f.readlines()[1::2]
        sequences = [sequence.upper() for sequence in sequences]
        # remove trailing '\n\'
        sequences = [sequence[:-1] for sequence in sequences]
    return sequences

def process_file(file_path):
    sequences = read_data(file_path)
    stats = [get_seq_embedding(s) for s in sequences]
    return stats

s1 = process_file("randoms1500")
s2 = process_file("vista1500")
X = s1 + s2
y = [0] * len(s1) + [1] * len(s2)

X = np.array(X)
y = np.array(y)

y[0], X[0]

(0, array([0.022     , 0.00533333, 0.012     , 0.018     , 0.00666667,
        0.002     , 0.00533333, 0.002     , 0.008     , 0.00266667,
        0.00733333, 0.00733333, 0.01666667, 0.00533333, 0.00733333,
        0.01133333, 0.00666667, 0.004     , 0.00933333, 0.006     ,
        0.006     , 0.00266667, 0.00066667, 0.        , 0.004     ,
        0.00133333, 0.004     , 0.00333333, 0.002     , 0.00066667,
        0.        , 0.00133333, 0.00666667, 0.00133333, 0.006     ,
        0.00466667, 0.00333333, 0.00066667, 0.00333333, 0.00066667,
        0.00266667, 0.00333333, 0.012     , 0.006     , 0.008     ,
        0.004     , 0.004     , 0.00733333, 0.01533333, 0.00666667,
        0.00666667, 0.00866667, 0.00733333, 0.002     , 0.004     ,
        0.        , 0.00266667, 0.00333333, 0.00733333, 0.00933333,
        0.01533333, 0.004     , 0.00333333, 0.004     , 0.00866667,
        0.004     , 0.00266667, 0.00466667, 0.00933333, 0.00133333,
        0.002     , 0.00066667, 0.004     , 0

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True)

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

# train one more on full data
clf = RandomForestClassifier()
_ = clf.fit(X, y)

precision    recall  f1-score   support

           0       0.91      0.35      0.51        60
           1       0.82      0.99      0.89       174

    accuracy                           0.82       234
   macro avg       0.86      0.67      0.70       234
weighted avg       0.84      0.82      0.79       234



In [6]:
import numpy as np

def read_chromosome(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()[1:]
        sequence = ''.join(lines)
        return sequence.upper().replace('\n','')

def split_chromosomes(sequence, step, window_size):
    splits = []
    for i in range(0, len(sequence) - window_size + 1, step):
        splits.append(sequence[i: i + window_size])

    return splits

chromosome = read_chromosome('chr21.fa')
splits = split_chromosomes(chromosome, 750, 1500)
splits[0]

'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN

In [7]:
def get_embeddings_with_status(sequences):
    status = np.zeros(len(sequences), np.bool)
    embeddings = []
    for i, seq in enumerate(sequences):
        if 'N' in seq:
            continue
        else:
            embeddings.append(get_seq_embedding(seq))
            status[i] = True
    return embeddings, status

embeddings, status = get_embeddings_with_status(splits)
embeddings[0], status[:50]

(array([0.01133333, 0.00333333, 0.00466667, 0.00466667, 0.00133333,
        0.00133333, 0.002     , 0.        , 0.00666667, 0.00466667,
        0.00333333, 0.00266667, 0.00466667, 0.00266667, 0.00333333,
        0.00466667, 0.00333333, 0.00133333, 0.00133333, 0.00933333,
        0.00533333, 0.00266667, 0.00333333, 0.00133333, 0.00133333,
        0.002     , 0.00266667, 0.00333333, 0.        , 0.00066667,
        0.00066667, 0.00266667, 0.00266667, 0.00266667, 0.002     ,
        0.00266667, 0.00466667, 0.00266667, 0.004     , 0.        ,
        0.00133333, 0.004     , 0.00933333, 0.00333333, 0.002     ,
        0.002     , 0.00466667, 0.00333333, 0.00333333, 0.00333333,
        0.00466667, 0.00533333, 0.00266667, 0.008     , 0.00666667,
        0.        , 0.00266667, 0.00333333, 0.00666667, 0.00666667,
        0.00666667, 0.00666667, 0.00666667, 0.006     , 0.00533333,
        0.00066667, 0.00666667, 0.00266667, 0.00333333, 0.00666667,
        0.00333333, 0.002     , 0.00066667, 0.00

In [17]:
partial_predict = clf.predict_proba(embeddings)[:,1]
avg = np.average(partial_predict)
partial_predict[:50], avg

(array([0.59, 0.66, 0.69, 0.75, 0.68, 0.73, 0.65, 0.67, 0.78, 0.76, 0.72,
        0.55, 0.77, 0.66, 0.7 , 0.69, 0.69, 0.83, 0.73, 0.8 , 0.83, 0.68,
        0.64, 0.71, 0.74, 0.83, 0.46, 0.42, 0.47, 0.68, 0.75, 0.72, 0.75,
        0.8 , 0.85, 0.83, 0.64, 0.63, 0.46, 0.5 , 0.64, 0.38, 0.62, 0.77,
        0.77, 0.73, 0.72, 0.79, 0.68, 0.43]), 0.5906005060444194)

In [18]:
def get_full_prediction(partial_predict, status, default_value):
    full_prediction = np.empty(len(status))
    full_prediction[status] = partial_predict
    full_prediction[~status] = default_value
    return full_prediction

full_pred = get_full_prediction(partial_predict, status, avg)
full_pred[:100]

array([0.59060051, 0.59060051, 0.59060051, 0.59060051, 0.59060051,
       0.59060051, 0.59060051, 0.59060051, 0.59060051, 0.59060051,
       0.59060051, 0.59060051, 0.59060051, 0.59060051, 0.59060051,
       0.59060051, 0.59060051, 0.59060051, 0.59060051, 0.59060051,
       0.59060051, 0.59060051, 0.59060051, 0.59060051, 0.59060051,
       0.59060051, 0.59060051, 0.59060051, 0.59060051, 0.59060051,
       0.59060051, 0.59060051, 0.59060051, 0.59060051, 0.59060051,
       0.59060051, 0.59060051, 0.59060051, 0.59060051, 0.59060051,
       0.59060051, 0.59060051, 0.59060051, 0.59060051, 0.59060051,
       0.59060051, 0.59060051, 0.59060051, 0.59060051, 0.59060051,
       0.59060051, 0.59060051, 0.59060051, 0.59060051, 0.59060051,
       0.59060051, 0.59060051, 0.59060051, 0.59060051, 0.59060051,
       0.59060051, 0.59060051, 0.59060051, 0.59060051, 0.59060051,
       0.59060051, 0.59060051, 0.59060051, 0.59060051, 0.59060051,
       0.59060051, 0.59060051, 0.59060051, 0.59060051, 0.59060

In [19]:
np.average(full_pred)

0.5906005060444194

In [20]:
def save_data(data, chrN, step, window_size):
    with open('output.wig', 'w+') as f:
        f.write(f"fixedStep chrom={chrN} start=0 step={step} span={window_size}\n")
        for x in data:
            f.write(f'{x}\n')

save_data(full_pred, 21, 750, 1500)